## Module 10 Assignment - Scraping a Website
* Author: brandon chiazza
* version 2.0

We will be creating a web scraper to parse a table from the Charities Bureau Website. From the website: “All 
charitable organizations operating in New York State are required by law to register and file annual financial reports 
with the Attorney General's Office. This includes any organization that conducts charitable activities, holds property 
that is used for charitable purposes, or solicits financial or other contributions.”

In [31]:
###Load mo webdriver-manager
!pip install selenium
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,"/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element(By.XPATH,"/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()
sleep(4) #allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements(By.CSS_SELECTOR,'tr'):
      cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
5,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
6,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
7,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
8,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
9,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA


In [32]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://yuchenjibucket/'#specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://yuchenjibucket/charities_bureau_scrape_20230513193213.csv


In [33]:
# remove first row of dataframe (None values)
df = df.drop(df.index[0])

# save the cleaned dataframe to a new CSV file
df.to_csv('charities_bureau_scrape_cleaned.csv', index=False)

# upload the cleaned CSV file to S3
s3 = boto3.resource('s3')
bucket_name = 'yuchenjibucket'
file_path = 'charities_bureau_scrape_cleaned.csv'
s3.meta.client.upload_file(file_path, bucket_name, file_path)
display(df)

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
5,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
6,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
7,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
8,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
9,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
10,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY


In [38]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'yuchenjibucket'
s3.create_bucket(Bucket=bucket_name)


{'ResponseMetadata': {'RequestId': 'CE86VWPD1BS42YQ3',
  'HostId': 'lUrLfLg9hnz/iB0PNZXGwItYhJ3xwm/rDisE0vUmj2mB00264IPPCdRTMFxYXxnDtU8u3FU1o9Y=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'lUrLfLg9hnz/iB0PNZXGwItYhJ3xwm/rDisE0vUmj2mB00264IPPCdRTMFxYXxnDtU8u3FU1o9Y=',
   'x-amz-request-id': 'CE86VWPD1BS42YQ3',
   'date': 'Tue, 02 May 2023 07:25:46 GMT',
   'location': '/yuchenjibucket',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/yuchenjibucket'}

In [59]:
!pip install webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,"/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element(By.XPATH,"/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()
sleep(4) #allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)

#####CREATE DATE FRAME#####
#create empty dataframe
df = pd.DataFrame(columns=["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])

In [74]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up web driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Navigate to website
driver.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

# Wait for search form to load
WebDriverWait(driver, 360).until(EC.presence_of_element_located((By.ID, 'searchForm')))

# Select "All Charities" radio button
all_charities = driver.find_element(By.ID, 'chtype_all')
all_charities.click()

# Submit search form
search_button = driver.find_element(By.ID, 'submitBtn')
search_button.click()

# Wait for search form to load
WebDriverWait(driver, 360).until(EC.presence_of_element_located((By.ID, 'searchForm')))

# Get total number of pages
try:
    pagination_element = driver.find_element(By.CLASS_NAME, 'pagination')
    num_pages = len(pagination_element.find_elements_by_tag_name('li')) - 2
except:
    num_pages = 1

# Loop through all pages of results
for i in range(num_pages):
    # Get table rows for current page
    table = driver.find_element(By.ID, 'charityTable')
    rows = table.find_elements_by_tag_name('tr')[1:]

    # Extract data from table rows
    for row in rows:
        columns = row.find_elements_by_tag_name('td')
        name = columns[0].text
        ein = columns[1].text
        city = columns[2].text
        state = columns[3].text

        # Do something with the data
        print(name, ein, city, state)

    # Navigate to next page of results
    if i < num_pages - 1:
        next_button = driver.find_element(By.XPATH, "//li[@class='next']/a")
        next_button.click()
        time.sleep(3)

# Close web driver
driver.quit()


KeyboardInterrupt: 

## References
* https://www.programiz.com/python-programming/working-csv-files
* https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
* https://realpython.com/python-boto3-aws-s3/
* https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 